# Spatial data integration with scRNA-seq
## Day 2 - Practical 3

#### 23rd January 2025
#### Ahmed Mahfouz, Benedetta Manzato (LUMC) 

### Agenda:
<font size="3">1. Dataset description

<font size="3">2. Spatial imputation with SpaGE and Tangram

<font size="3">3. Cell deconvolution with Cell2location

<font size="3">4. Access image in AnnData object

# Dataset: MERFISH spatial transcriptomics dataset of a single adult mouse brain (Zhuang-ABCA-1) (Xiaowei Zhuang)

<font size="3"> This spatially resolved cell atlas of the whole mouse brain provides a systematic characterization of the spatial organization of transcriptomically defined cell types across the entire adult mouses brain using in situ, single-cell transcriptomic profiling with multiplexed error-robust fluorescence in situ hybridization (MERFISH). A gene panel of 1,122 genes was imaged in 9 million brain cells using **MERFISH**, and spatially resolved, single-cell expression profiling was performed at the whole-transcriptome scale by integrating the MERFISH data with the whole-brain single-cell RNA-sequencing (scRNA-seq) dataset previously generated by the Allen Institute. This resulted in the generation of a comprehensive cell atlas of more than 5,000 transcriptionally distinct cell clusters, belonging to ∼300 major cell types (subclasses), across the whole mouse brain, with high molecular and spatial resolution.
The authors performed MERFISH imaging on **245 coronal and sagittal sections**, the cell atlas was then spatially registered to the Allen Mouse Brain Common Coordinate Framework (CCFv3), which allows systematic quantifications of the cell composition and organization in individual brain regions as anatomically delineated in the CCFv3. This cell atlas also reveals molecularly-defined brain regions characterized by distinct cell type compositions, spatial gradients featuring gradual changes in the gene-expression profiles of cells, as well as cell-type-specific cell-cell interactions and the molecular basis and functional implications of these cell-cell interactions.

<font size="3"> The collection spans four mouse specimens (two coronal sets and two sagittal sets):
- <font size="3">**Zhuang-ABCA-1**: 
  - **Dataset size**: 4.2 million cells spanning 147 coronal sections with a 1,122 gene panel.
  - **Filtered dataset**: 2.8 million cells passed the cell classification confidence score threshold and are displayed in the ABC atlas.
  
- <font size="3">**Zhuang-ABCA-2**: 
  - **Dataset size**: 1.9 million cells spanning 66 coronal sections.
  - **Filtered dataset**: 1.2 million cells passed the cell classification confidence score threshold.
  
- <font size="3">**Zhuang-ABCA-3**: 
  - **Dataset size**: 2.1 million cells spanning 23 sagittal sections.
  - **Filtered dataset**: 1.6 million cells passed the confidence score threshold.

- <font size="3">**Zhuang-ABCA-4**: 
  - **Dataset size**: 0.22 million cells spanning 3 sagittal sections.
  - **Filtered dataset**: 0.16 million cells passed the confidence score threshold.


<font size="3"> We will work with the Brain Coronal 1 collection (Zhuang-ABCA-1); the cell-by-gene matrix of the 8.4 millions cells can be downloaded from the AWS bucket of this animal. The cells are then filtered by cell-classification (label transfer) confidence scores calculated during MERFISH-scRNAseq data integration. 7.0 million cells passed the confidence score threshold for cell subclass label transfer and 5.8 million cells further passed the confidence score threshold for cell cluster label transfer. These 5.8 million cells are included in the cell metadata file that can be downloaded from the the AWS bucket and are displayed on the ABC Atlas. The CCF coordinates of the 5.4 million cells that were registered to the 3D Allen-CCF can be downloaded from the CCF coordinate files in the AWB bucket. The collection spans four mouse specimens (2 coronal sets and 2 sagittal sets). Cells are mapped to the whole mouse brain taxonomy (WMB-taxonomy) and Allen Common Coordinate Framework (Allen-CCF-2020). 
Refer to Zhang et al, 2023 for more details.


For more details: [Molecularly defined and spatially resolved cell atlas of the whole mouse brain (Zhang et al. 2023)](https://www.nature.com/articles/s41586-023-06808-9)

From https://github.com/AllenInstitute/abc_atlas_access/blob/main/notebooks/zhuang_merfish_tutorial.ipynb

Data Availablity : https://knowledge.brain-map.org/data/5C0201JSVE04WY6DMVC/collections </font>

In [ ]:
import time

# Record the start time
start_time = time.time()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Importing necessary libraries for spatial data analysis and visualization
import os, re
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np
import anndata as ad

# Display the version of important packages for reproducibility
sc.logging.print_header()

# Set the default aesthetics for Scanpy plots
sc.set_figure_params(facecolor='white', figsize=(8, 8))  # White background, figure size 8x8
sc.settings.verbosity = 0  # Set verbosity to show only errors (0) # errors (0), warnings (1), info (2), hints (3)

<font size="3"> We create an output folder in your local space to save outputs:

In [ ]:
folder_path = './outputs/'
if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder created: {folder_path}")
else:
    print(f"Folder already exists: {folder_path}")

In [ ]:
data_dir = '/data/spatial_workshop/day2/practical_3'

<font size="3">We will use the **Zhuang-ABCA-1 dataset**: 4.2 million cell spatial transcriptomics dataset spanning 147 coronal sections with a 1122 gene panel. 2.8 million cells passed cell classification confidence score threshold and displayed in the ABC atlas.

<font size="3">We decided to work with **one section only**, the one that captures most cells (section 80).

<img src="img/ABCAtlas_Zhuang-ABCA-1_Coronal Grid.png" width="800" height="300">

### Read data 

<font size="3">We will read the AnnData object that has been filtered to have only section 80 and QC was performed from the authors of the study.

In [ ]:
adata_section = ad.read_h5ad(f"{data_dir}/abc_download_root/expression_matrices/Zhuang-ABCA-1/adata_section80.h5ad")

<font size="3">Spatial coordinates, being metadata about cells/spots are stored either in **`.obs`** or **`.obsm`**

<font size="3">Spot metadata also contains info about class, subclass, neurotransmitter, patient metadata and section.

In [ ]:
adata_section.obs.head()

<font size="3"> Some spatial tools require the spatial coordinates can be stored as `spatial` or `X_spatial`

In [ ]:
# Add X_spatial to the right field
x_spatial = np.array(adata_section.obs[['x','y']])

adata_section.obsm['X_spatial'] = x_spatial
adata_section.obsm['X_spatial']

<font size="3">Gene information are stored in **`.var`**

In [ ]:
adata_section.var.head()

<font size="3">Plot the section and color them by all the different classifications avaiable in the metadata (class_color, subclass_color, supertype_color, cluster_color):

In [ ]:
# Create the color mapping dictionaries
class_palette = dict(zip(adata_section.obs['class'].unique(), adata_section.obs['class_color'].unique()))
subclass_palette = dict(zip(adata_section.obs['subclass'].unique(), adata_section.obs['subclass_color'].unique()))
supertype_palette = dict(zip(adata_section.obs['supertype'].unique(), adata_section.obs['supertype_color'].unique()))
cluster_palette = dict(zip(adata_section.obs['cluster'].unique(), adata_section.obs['cluster_color'].unique()))

# Create the subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 15))
fig.suptitle('Scatter Plots of Different Categories', fontsize=16)

# Plot 'class' using class_color
sns.scatterplot(ax=axes[0, 0], x='x', y=-adata_section.obs['y'], hue='class', palette=class_palette, data=adata_section.obs, s=8, legend=False)
axes[0, 0].set_title('Class')
axes[0, 0].set_xlabel('X Coordinate')
axes[0, 0].set_ylabel('Y Coordinate')
axes[0, 0].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
axes[0, 0].grid(False)  # Remove grid

# Plot 'subclass' using subclass_color
sns.scatterplot(ax=axes[0, 1], x='x', y=-adata_section.obs['y'], hue='subclass', palette=subclass_palette, data=adata_section.obs, s=8, legend=False)
axes[0, 1].set_title('Subclass')
axes[0, 1].set_xlabel('X Coordinate')
axes[0, 1].set_ylabel('Y Coordinate')
axes[0, 1].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
axes[0, 1].grid(False)  # Remove grid

# Plot 'supertype' using supertype_color
sns.scatterplot(ax=axes[1, 0], x='x', y=-adata_section.obs['y'], hue='supertype', palette=supertype_palette, data=adata_section.obs, s=8, legend=False)
axes[1, 0].set_title('Supertype')
axes[1, 0].set_xlabel('X Coordinate')
axes[1, 0].set_ylabel('Y Coordinate')
axes[1, 0].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
axes[1, 0].grid(False)  # Remove grid

# Plot 'cluster' using cluster_color
sns.scatterplot(ax=axes[1, 1], x='x', y=-adata_section.obs['y'], hue='cluster', palette=cluster_palette, data=adata_section.obs, s=8, legend=False)
axes[1, 1].set_title('Cluster')
axes[1, 1].set_xlabel('X Coordinate')
axes[1, 1].set_ylabel('Y Coordinate')
axes[1, 1].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
axes[1, 1].grid(False)  # Remove grid

plt.tight_layout(rect=[0, 0, 1, 0.96])  
plt.show()


<font size="3"> Perform PCA and UMAP and cluster with Leiden Algorithm

In [ ]:
sc.pp.pca(adata_section)
sc.pp.neighbors(adata_section)
sc.tl.umap(adata_section)
sc.tl.leiden(
    adata_section, 
    key_added="clusters", 
    resolution=1.0,
    directed=False, 
    n_iterations=2
)

In [ ]:
plt.rcParams["figure.figsize"] = (8,8)
sc.pl.umap(adata_section, color=["clusters"], wspace=0.2)

<font size="3"> Since the spatial coordinates are stored in  `.obsm[X_spatial]` you can plot the section with `sc.pl.spatial`


In [ ]:
# Generate the spatial plot
sc.pl.spatial(adata_section, color='class', color_map='Accent_r', spot_size=0.04, title='Section 80 colored by Class')

In [ ]:
# Subset the AnnData object to keep only cells belonging to a specific class
# and plot them again with sc.pl.spatial
adata_section_one_class = adata_section[adata_section.obs['class']=='01 IT-ET Glut']
sc.pl.spatial(adata_section_one_class, color='class', color_map='Accent_r', spot_size=0.04, title='Section 80 (one Class)')

### Plot gene expression in space

<font size="3">

Since each spot in the spatial data corresponds to a specific location and its associated gene expression profile, we can visualize spatial gene expression.

We can do this using the function `sc.pl.spatial`, it requires the name of the variable index to indicate the genes to plot.

Let's take a look at `adata_section`.var:

In [ ]:
adata_section.var.head()

<font size="3">

Change the index of the `adata_section.var` to `gene_symbol` for clarity

In [ ]:
# Reset the index to turn the current index into a column
adata_section.var.reset_index(inplace=True)
adata_section.var.set_index('gene_symbol', inplace=True)
adata_section.var.head()

In [ ]:
# Define genes to plot
genes_toplot = ['Zic1',
 'Penk',
 'Gfap',
 'Nr4a2',
 'Satb2']

sc.pl.spatial(adata_section, color=genes_toplot,spot_size=0.04)

<font size="3"> Plot your favourite gene/s

In [ ]:
#genes_toplot = [.....]

#sc.pl.spatial(adata_section, color=genes_toplot,spot_size=0.04)

In [ ]:
# Record the end time
end_time = time.time()

# Calculate elapsed time in minutes
elapsed_time = (end_time - start_time) / 60
print(f"Execution Time: {elapsed_time:.2f} minutes")

In [ ]:
import gc
# collect garbage
gc.collect()

# Prediction of spatial patterns for unmeasured genes using scRNA-seq data with SpaGE
[SpaGE: Spatial Gene Enhancement using scRNA-seq (Abdelaal et al. 2020)](https://academic.oup.com/nar/article/48/18/e107/5909530)

In [ ]:
start_time = time.time()

In [ ]:
from SpaGE.main import SpaGE
import scipy.stats as st

<figure>
    <img src="img/spage.jpg" width="700" height="280" alt="Description of the image">
    <figcaption>From <a href="https://academic.oup.com/nar/article/48/18/e107/5909530" target="_blank">Abdelaal et al. (2020) NAR</a></figcaption>
</figure>

<font size="3"> SpaGE takes as input the (targeted) spatial dataset (query) and a single-cell RNA dataset used as reference. The two datasets should be generated under the same conditions (same species, tissue, region) as much as possible.

<font size="3"> We will use [Mouse whole-brain transcriptomic cell type atlas (Hongkui Zeng)](https://www.nature.com/articles/s41586-023-06812-z).This scRNAseq dataset profiled 7 million cells (approximately 4.0 million cells passing quality control). More in detail:

 - <font size="3"> 1.7 million single cell transcriptomes spanning the whole adult mouse brain using 10Xv2 chemistry (WMB-10Xv2)

 - <font size="3"> 2.3 million single cell transcriptomes spanning the whole adult mouse brain using 10Xv3 chemistry (WMB-10Xv3)

 - <font size="3"> 1687 single cell transcriptomes spanning the whole adult mouse brain using the 10X Multiome chemistry (WMB-10XMulti)

<figure>
    <img src="img/sc.png" width="700" height="280" alt="Description of the image">
    <figcaption>From <a href="https://www.nature.com/articles/s41586-023-06812-z" target="_blank">Yao et al. (2023) Nature Methods</a></figcaption>
</figure>


<font size="3"> The size is of the combined single-cell object from AbcProjectCache is around 280GB. For this reason, the three objects have been previously downloaded, concatenated, preprocessed and subsetted to obtain NNN cells. For your knowledge, you can find all the steps in `obtain_scref.py`.

<font size="3"> Let's read the resulting single-cell AnnData object:

In [ ]:
adata_sc = ad.read_h5ad(f"{data_dir}/SpaGE_data/sc_adata_1percent.h5ad")

<font size="3"> Inspect the object:

In [ ]:
adata_sc

In [ ]:
adata_sc.obs.head()

<font size="3"> Add feature metadata to the AnnData object

In [ ]:
genes = pd.read_csv(f"{data_dir}/abc_download_root/metadata/WMB-10X/gene.csv",index_col=0)

genes.head()

In [ ]:
adata_sc.var = genes

In [ ]:
# Reset the index to turn the current index into a column
adata_sc.var.reset_index(inplace=True)
adata_sc.var.set_index('gene_symbol', inplace=True)
adata_sc.var.head()

In [ ]:
# collect garbage
gc.collect()

<font size="3"> Set up inputs for SpaGE:

In [ ]:
# Extract the count data and transform them into a Pandas DataFrame
RNA_data = pd.DataFrame(adata_sc.X.todense())

In [ ]:
# Add gene names to the RNA_data columns 
RNA_data.columns = adata_sc.var.index

In [ ]:
# Filter lowely expressed genes
Genes_count = np.sum(RNA_data > 0, axis=1)
RNA_data = RNA_data.loc[Genes_count >=10,:]
del Genes_count

In [ ]:
# Normalization
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
RNA_data = RNA_data.apply(Log_Norm_cpm,axis=0)

In [ ]:
# transpose RNA_data to have rows = genes and columns = cells
RNA_data = RNA_data.T

In [ ]:
RNA_data.head()

In [ ]:
# Drop any row with at least one NaN value
RNA_data = RNA_data.dropna()  # This will drop any row with at least one NaN value

<span style="color:; font-size: 15px;"> **How many genes do the spatial and single-cell data set have in common?**

<span style="color:; font-size: 15px;"> **How many genes are measured in the single-cell dataset that are not measured in the spatial dataset?**

In [ ]:
## use .intersection
gene_intersection = set(list(adata_section.var.index)).intersection(set(list(adata_sc.var.index)))

## use a set operation
genes_not_intersection = list(set(RNA_data.index) - set(adata_section.var.index))

print(f"Number of genes measured both in the sc and spatial datasets: {len(gene_intersection)}")
print(f"Number of genes measured in the sc reference only: {len(genes_not_intersection)}")

In [ ]:
# Extract count matrix from adata_section and rename index and column names

spatial_data = pd.DataFrame(adata_section.X)
spatial_data.index = adata_section.obs.index
spatial_data.columns = adata_section.var.index

In [ ]:
# Normalize spatial_data 
spatial_data = spatial_data.apply(Log_Norm_cpm,axis=0)

In [ ]:
brain_region_specific_genes = [
    "Bcl11b", "Cux2", "Fezf2", "Tbr1", "Satb2", # isocortex
    "Foxp2",  # Striatum specific
    "Penk",   # Basal ganglia specific
    "Zic1",   # Cerebellum specific
    "Hoxb2",  # Hindbrain specific
    "Dlx6",   # Subpallium specific
]

# Checking for genes from brain_region_specific_genes that are measured by spatial data
overlap_genes = list(set(list(spatial_data.columns)) & set(brain_region_specific_genes))
overlap_genes

In [ ]:
# Checking for genes from brain_region_specific_genes that are NOT measured by spatial data
non_overlap_genes = list(set(brain_region_specific_genes) - set(overlap_genes))

### Leave-one-gene-out cross validation

<font size="3"> Define the function to predict genes measured in the spatial dataset to have a ground truth and plot the measured expression next to the predicted one.

```python
def predicted_GE_LOOCV(Gene_set, spatial_data, RNA_data):
    
    # Initialize an empty DataFrame to store the predicted values
    predicted_df = pd.DataFrame(index=spatial_data.index)

    for i in Gene_set:
        # Predict the expression of the current gene using the SpaGE method
        Imp_Genes = SpaGE(spatial_data.drop(i, axis=1), RNA_data.T, n_pv=30, genes_to_predict=[i])
        Imp_Genes.index = spatial_data.index
        
        # Add the predicted values for the current gene as a new column in the DataFrame
        predicted_df[i] = Imp_Genes[i]

    return predicted_df

In [ ]:
predicted_df = pd.read_csv(f"{data_dir}/SpaGE_data/predicted_GE_loocv_SpaGE.csv",index_col=0)

<font size="3"> Since we have a ground truth, create a Series to store Spearman correlation results for each gene.

<font size="3"> Our gene_set in this case is a set of genes shared between the spatial and the sc datasets.

In [ ]:
import matplotlib.gridspec as gridspec  # For precise subplot layout control
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

plt.style.use('dark_background')

def plot_measured_vs_predicted_spatial(Gene_set, spatial_data, predicted_df, adata_section):
    """
    Plot measured and predicted spatial expression for genes, with calculated Spearman correlations.
    
    Parameters:
        Gene_set (list): List of genes to visualize.
        spatial_data (DataFrame): Measured expression data (columns are genes, rows are spots).
        predicted_df (DataFrame): Predicted expression data (columns are genes, rows are spots).
        adata_section (AnnData): Spatial data with 'x' and 'y' coordinates.
    
    Returns:
        dict: Spearman correlations for each gene.
    """
    correlations = {}  # Dictionary to store Spearman correlations
    
    for gene in Gene_set:
        # Calculate Spearman correlation
        correlation = spearmanr(spatial_data[gene], predicted_df[gene])[0]
        correlations[gene] = correlation
        
        # Create a figure with two scatterplots
        fig = plt.figure(figsize=(15, 8))  # Adjust figure size
        gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)  # Create grid spec for layout
        
        # --- Measured Gene Expression ---
        ax0 = fig.add_subplot(gs[0])
        cmap_measured = spatial_data[gene]
        cmap_measured = np.clip(cmap_measured, None, np.percentile(cmap_measured, 99))  # Clip to 99th percentile
        sc1 = ax0.scatter(adata_section.obs['x'], -adata_section.obs['y'], s=1, c=cmap_measured, cmap="viridis", alpha=1)
        
        ax0.set_title(f"Measured Expression: {gene}", fontsize=12)
        ax0.set_xlabel("X Coordinate")
        ax0.set_ylabel("Y Coordinate")
        ax0.set_aspect("equal", adjustable="box")
        ax0.grid(False)
        
        # --- Predicted Gene Expression ---
        ax1 = fig.add_subplot(gs[1])
        cmap_predicted = predicted_df[gene]
        cmap_predicted = np.clip(cmap_predicted, None, np.percentile(cmap_predicted, 99))  # Clip to 99th percentile
        sc2 = ax1.scatter(adata_section.obs['x'], -adata_section.obs['y'], s=1, c=cmap_predicted, cmap="viridis", alpha=1)
        
        ax1.set_title(f"Predicted Expression: {gene}", fontsize=12)
        ax1.set_xlabel("X Coordinate")
        ax1.set_ylabel("Y Coordinate")
        ax1.set_aspect("equal", adjustable="box")
        ax1.grid(False)
        
        # --- Colorbar ---
        cbar_ax = fig.add_subplot(gs[2])  # Third subplot for color bar
        cbar = fig.colorbar(sc1, cax=cbar_ax)
        cbar.set_label("Expression")
        
        # Adjust layout and show
        plt.tight_layout()
        plt.show()
    
    return correlations


In [ ]:
correlations = plot_measured_vs_predicted_spatial(overlap_genes, spatial_data, predicted_df, adata_section)

<font size="3"> Let's take a look at the Spearman Correlations between measured and predicted gene expression:

In [ ]:
# Since we have a ground truth, create a Series to store Spearman correlation results for each gene
# Our gene_set in this case is a set of genes shared between the spatial and the sc datasets
correlations_spage = pd.DataFrame(list(correlations.items()), columns=["Gene", "correlations_spage"]).set_index("Gene")

In [ ]:
correlations_spage

### Predict unmeasured genes

<font size="3">  Define a function to predict unmeasured genes in the spatial dataset. 

<font size="3">  In this case correlations cannot be calculated as there is no ground truth available.

```python

def predicted_unmeasured_GE(Gene_set, spatial_data, RNA_data):
    
    # Initialize an empty DataFrame to store the predicted values
    predicted_df = pd.DataFrame(index=spatial_data.index)

    for i in Gene_set:
        # Predict the expression of the current gene using the SpaGE method
        Imp_Genes = SpaGE(spatial_data, RNA_data.T, n_pv=30, genes_to_predict=[i])
        Imp_Genes.index = spatial_data.index
        
        # Add the predicted values for the current gene as a new column in the DataFrame
        predicted_df[i] = Imp_Genes[i]

    return predicted_df

In [ ]:
predicted_df = pd.read_csv(f"{data_dir}/SpaGE_data/predicted_unmeasured_GE_SpaGE.csv",index_col=0)

In [ ]:
def plot_predicted_spatial(Gene_set, predicted_df, adata_section):
    """
    Plot predicted spatial expression for genes.
    
    Parameters:
        Gene_set (list): List of genes to visualize.
        predicted_df (DataFrame): Predicted expression data (columns are genes, rows are spots).
        adata_section (AnnData): Spatial data with 'x' and 'y' coordinates.
    
    Returns:
        None
    """
    for gene in Gene_set:
        # Create a figure for the predicted expression
        fig, ax = plt.subplots(figsize=(8, 6))  # Adjust figure size
        
        # Get predicted values and clip to the 99th percentile
        cmap_predicted = predicted_df[gene]
        cmap_predicted = np.clip(cmap_predicted, None, np.percentile(cmap_predicted, 99))
        
        # Plot the predicted expression
        scatter = ax.scatter(
            adata_section.obs['x'], -adata_section.obs['y'], 
            s=1, c=cmap_predicted, cmap="viridis", alpha=1
        )
        
        # Add title and labels
        ax.set_title(f"Predicted Expression: {gene}", fontsize=14)
        ax.set_xlabel("X Coordinate")
        ax.set_ylabel("Y Coordinate")
        ax.set_aspect("equal", adjustable="box")
        ax.grid(False)
        
        # Add a colorbar
        cbar = plt.colorbar(scatter, ax=ax, pad=0.01)
        cbar.set_label("Expression")
        
        # Adjust layout and show
        plt.tight_layout()
        plt.show()

In [ ]:
# Define the set of genes (from the non_overlap_genes list) to be predicted by SpaGE
plot_predicted_spatial(non_overlap_genes, predicted_df, adata_section)

<font size="3"> There is no definitive ground truth for the real gene expression profiles of these genes. However, an alternative approach could be to plot region-specific genes and verify whether their expression aligns with the expected brain regions.

<font size="3"> For example, Tbr1 is known to be Isocortex-specific; does its expression show higher levels in the Isocortex? Similarly, Foxp2 is Striatum-specific; does it exhibit a Striatum-specific expression pattern?

<font size="3"> Use [this](https://mouse.brain-map.org/search/index) resource to check region-specific gene expression.

In [ ]:
# Record the end time
end_time = time.time()

# Calculate elapsed time in minutes
elapsed_time = (end_time - start_time) / 60
print(f"Execution Time SpaGE: {elapsed_time:.2f} minutes")

<font size="3"> Let's delete some objects we won't be using anymore to free some RAM:

In [ ]:
del RNA_data, spatial_data

In [ ]:
# collect garbage
gc.collect()

# Unmeasured gene imputation with Tangram
[Deep learning and alignment of spatially resolved single-cell transcriptomes with Tangram (Biancalani et al. 2021)](https://www.nature.com/articles/s41592-021-01264-7)

In [ ]:
start_time = time.time()

In [ ]:
import tangram as tg

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor="white")

#### Common gene set between reference and spatial dataset

<font size="3"> We want to select our training genes. These genes are shared between the two datasets and should capture the biological variance between cell types. For this, we first compute marker genes on the single-cell data and then use Tangram’s preprocessing function to subset to those genes that are also present in the spatial data.

In [ ]:
markers = list(set.intersection(set(adata_sc.var_names), set(adata_section.var_names)))
len(markers)

<font size="3"> `tg.pp_adatas` computes the overlap between single-cell data and spatial data on the list of genes provided in the genes argument, it then stores the resulting gene set under 'training_genes' in both adata objects under the `.uns` key and enforces consistent ordering of the genes.

In [ ]:
tg.pp_adatas(adata_sc, adata_section, genes=markers) # using the same single-cell reference that we used to run SpaGE

<font size="3"> Check that the function performs as we expect:

In [ ]:
assert "training_genes" in adata_sc.uns
assert "training_genes" in adata_section.uns

print(f"Number of training_genes: {len(adata_sc.uns['training_genes'])}")

#### Computing the map from single-cells to spatial voxels

<font size="3"> Having specified the training genes, we can now create the map from dissociated single-cell measurements to the spatial locations. For this, we are going to use the `map_cells_to_space` function from tangram. This function has two different modes, `mode='cells'` and `mode='clusters'`. The latter only maps averaged single-cells which makes the mapping computationally faster and more robust when mapping between specimen.

```python
ad_map = tg.map_cells_to_space(
    adata_sc,
    adata_section,
    mode="cells",
    density_prior="rna_count_based",
    num_epochs=500,
    device="cpu",  # or: cpu
)


<font size="3"> We run `tg.map_cells_to_space` for you (takes a while to run), let's just read the resulting AnnData object

In [ ]:
ad_map = ad.read_h5ad(f"{data_dir}/Tangram_data/ad_map_tangram.h5ad")
ad_map

<font size="3"> We observe that Tangram’s mapping from cell i to spatial voxels j is stored in the .X property of ad_map.
<font size="3">  The meaning of the .var and .obs also changes:

- <font size="3"> in `.var` we have the available metadata of the spatial data, adata_section
- <font size="3"> in `.obs` we have the available metadata of the single-cell data, adata_sc
- <font size="3"> In addition, the information about the training run is stored in the .uns key, see `.uns['training_genes_df']` and `.uns[training_history]`.

<font size="3"> Now we can **project the genes present in the single-cell data to the spatial locations**. This is easily achieved by multiplying the mapping matrix stored in ad_map with the original single-cell data stored in adata_sc. Tangram already provides a convenience function which takes in a mapping and the corresponding single-cell data. The result is a **spatial voxel by genes matrix** which technically is identical to the original spatial data adata_section but **contains expression values for all genes**.

```python
ad_ge = tg.project_genes(adata_map=ad_map, adata_sc=adata_sc)
ad_ge

<font size="3"> We run `tg.project_genes` for you, let's just read the resulting AnnData object

In [ ]:
ad_ge = ad.read_h5ad(f"{data_dir}/Tangram_data/ad_ge.h5ad")

## add .obsm['X_spatial'] so we can use sc.pl.spatial to plot gene expression
ad_ge.obsm['X_spatial'] = np.array(ad_ge.obs[['x','y']])
ad_ge

<font size="3"> Next, we will compare the new spatial data with the original measurements.

<font size="3"> To do this we will plot the true expression and the predicted expression of the shared genes, as well as the Spearman correlation between the true and predicted expression in all the spots.

In [ ]:
# Use the same set of genes that we predicted with SpaGE
# we need to make the genes all lower-case first
overlap_genes = [gene.lower() for gene in overlap_genes]
overlap_genes

<font size="3"> Plot the true expression from adata_section

In [ ]:
sc.pl.spatial(adata_section, color=overlap_genes, spot_size=0.04)

<font size="3"> Plot the predicted expression from ad_ge

In [ ]:
ad_ge.var.reset_index(inplace=True)
ad_ge.var.set_index('_index', inplace=True)

In [ ]:
sc.pl.spatial(ad_ge, color=overlap_genes, spot_size=0.04)

<font size="3"> Calculate Spearman correlations

In [ ]:
# Subset the AnnData objects to include only the genes in overlap_genes
# Assuming both AnnData objects have the same gene ordering
ad_true  = adata_section[:, adata_section.var_names.isin(overlap_genes)]
ad_ge_predicted = ad_ge[:, ad_ge.var_names.isin(overlap_genes)]

# Initialize an empty dictionary to store correlations
correlation_dict = {}

# Loop over each gene to calculate Spearman correlation
for gene in overlap_genes:
    # Extract gene expression vectors for the current gene
    expr1 = ad_true[:, gene].X.flatten()
    expr2 = ad_ge_predicted[:, gene].X.flatten()
    
    # Calculate Spearman correlation for the current gene
    corr, _ = st.spearmanr(expr1, expr2)
    
    # Store the result in the dictionary
    correlation_dict[gene] = corr

# Convert the dictionary to a DataFrame
tangram_correlations = pd.DataFrame.from_dict(correlation_dict, orient='index', columns=['correlations_tangram'])
# capitalize gene names! 
tangram_correlations.index = tangram_correlations.index.str.capitalize()
tangram_correlations

<font size="3"> Since we predicted the same genes as SpaGE, let's compare the Spearman correlations:

In [ ]:
all_correlations = pd.concat([correlations_spage, tangram_correlations],axis=1)
all_correlations[['correlations_spage','correlations_tangram']]

<font size="3"> **Which tool results in higher correlation between the true-measured gene expression and the predicted expression of the genes of interest?**

In [ ]:
# Record the end time
end_time = time.time()

# Calculate elapsed time in minutes
elapsed_time = (end_time - start_time) / 60
print(f"Execution Time Tangram: {elapsed_time:.2f} minutes")

<font size="3"> Let's delete some objects we won't be using anymore to free some RAM:

In [ ]:
del ad_map, ad_ge

In [ ]:
del all_correlations, correlations_spage, tangram_correlations, predicted_df, ad_true, ad_ge_predicted

In [ ]:
gc.collect()

# Cell type deconvolution with cell2location
[Cell2location maps fine-grained cell types in spatial transcriptomics (Kleshchevnikov et al. 2022)](https://www.nature.com/articles/s41587-021-01139-4)

In [ ]:
start_time = time.time()

In [ ]:
#import squidpy as sq
import cell2location as c2l

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor="white")

In [ ]:
# set raw layer as X
adata_sc.layers['raw'] = adata_sc.X

<font size="3"> Add metadata with cell type annotation to the single-cell AnnData object

In [ ]:
# read csv
metadata = pd.read_csv(f"{data_dir}/abc_download_root/metadata/WMB-10X/views/cell_metadata_with_cluster_annotation.csv",index_col=0)
# filter metadata to keep only obervations in the subsetted SC AnnData object
metadata = metadata.loc[metadata.index.intersection(adata_sc.obs.index)]
# reindex rows
metadata = metadata.reindex(adata_sc.obs.index)
# assign metadata to adata_sc.obs (in this way metadata replaces all fields previously in obs)
adata_sc.obs = metadata

adata_sc.obs.head()

<font size="3"> Identify the shared set of genes between spatial and single-cell 

In [ ]:
shared_features = [feature for feature in adata_section.var_names if feature in adata_sc.var_names]

# filter 
adata_sc = adata_sc[:, adata_sc.var.index.isin(shared_features)].copy()
adata_section = adata_section[:, shared_features].copy() # probably no need to filter the spatial gene set
# reorder genes to have them match between spatial data and sc
adata_sc = adata_sc[:, shared_features].copy()

adata_section.shape[1] == adata_sc.shape[1]
print(f'Number of shared genes between spatial data and reference sc: {adata_section.shape[1]}')

<font size="3"> **Fit the reference model**

<font size="3"> When your spatial transcriptomics dataset is whole-transcriptome: The default parameters cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12 are a good starting point, however, you can increase the cut-off to exclude more genes. To preserve marker genes of rare cell types we recommend low cell_count_cutoff=5, however, cell_percentage_cutoff2 and nonz_mean_cutoff can be increased to select between 8k-16k genes.

<font size="3"> In this 2D histogram, orange rectangle highlights genes excluded based on the combination of number of cells expressing that gene (Y-axis) and average RNA count for cells where the gene was detected (X-axis).

<font size="3"> This step can be skipped when a limited gene panel is available.

In [ ]:
# fit the reference model
selected = c2l.utils.filtering.filter_genes(
    adata_sc, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12
)

adata_sc = adata_sc[:, selected].copy()
adata_section = adata_section[:, selected].copy()

In [ ]:
print(f"The spatial AnnData object has {adata_section.shape[0]} obs and {adata_section.shape[1]} features.")

In [ ]:
print(f"The single-cell AnnData object has {adata_sc.shape[0]} obs and {adata_sc.shape[1]} features.")

<font size="3"> **Estimation of reference cell type signatures (NB regression)**

<font size="3"> The following two command are computationally intensive and require GPU. We run them for you. You can read the output from `stdata_deconv.h5ad`

<font size="3"> The signatures are estimated from scRNA-seq data, accounting for batch effect, using a Negative binomial regression model.

<font size="3"> First, prepare anndata object for the regression model:

```python
c2l.models.RegressionModel.setup_anndata(
    adata=adata_sc,
    #batch_key="Replicates", # there's no batch 
    labels_key="class", # cell type = class
    #categorical_covariate_keys=["assay"],
    layer="raw",
)

```python
model = c2l.models.RegressionModel(adata_sc)
# default, try on GPU:
model.train(max_epochs=250, batch_size=2500, train_size=1, lr=0.002)

model.export_posterior(
    adata_sc,
    sample_kwargs={"num_samples": 1000, "batch_size": 2500},
)

<font size="3"> **Train the model** to estimate the reference cell type signatures:

```python
# export estimated expression in each cluster
if "means_per_cluster_mu_fg" in adata_sc.varm.keys():
    inf_aver = adata_sc.varm["means_per_cluster_mu_fg"][
        [f"means_per_cluster_mu_fg_{i}" for i in adata_sc.uns["mod"]["factor_names"]]
    ].copy()
else:
    inf_aver = adata_sc.var[
        [f"means_per_cluster_mu_fg_{i}" for i in adata_sc.uns["mod"]["factor_names"]]
    ].copy()

inf_aver.columns = adata_sc.uns["mod"]["factor_names"]
inf_aver.head()

# cell 2 location cell type mapping
print('start with Cell2location')
c2l.models.Cell2location.setup_anndata(
    adata=adata_section,
)

model = c2l.models.Cell2location(
    adata_section,
    cell_state_df=inf_aver,
    N_cells_per_location=8,
)
model.view_anndata_setup()

print('start with training')
model.train(max_epochs=30000, batch_size=None, train_size=1)#, use_gpu=use_gpu)

adata_section = model.export_posterior(
    adata_section,
    sample_kwargs={
        "num_samples": 1000,
        "batch_size": model.adata.n_obs}#,
        #"use_gpu": use_gpu,
)


adata_section.obs[adata_section.uns["mod"]["factor_names"]] = adata_section.obsm["q05_cell_abundance_w_sf"]

<font size="3"> Read the pre-computed AnnData object with cell2location results

In [ ]:
adata_section = ad.read_h5ad(f"{data_dir}/cell2location_data/stdata_deconv.h5ad")

In [ ]:
### cell2location result
adata_section.obs[adata_section.obs['class'].unique()]

<font size="3"> Visualize cell abundance in spatial coordinates:

In [ ]:
sc.pl.spatial(adata_section, color=['01 IT-ET Glut','04 DG-IMN Glut','30 Astro-Epen'],spot_size=0.04)

In [ ]:
# Record the end time
end_time = time.time()

# Calculate elapsed time in minutes
elapsed_time = (end_time - start_time) / 60
print(f"Execution Time cell2location: {elapsed_time:.2f} minutes")

In [ ]:
# collect garbage
gc.collect()